In [3]:
import numpy as np
import sympy as sp
import functions as fn
from IPython.display import display, Math
import scipy.integrate as int


"""
Definición de operadores para dos sistenas de referencia distintos, y una base conjunta que pueda ser autofunción de tales operadores
"""

#Definimos parametros a usar en unidades atómicas
hbar = 1
me   = 1

w, phi, theta , phi_p,theta_p = sp.symbols('w phi theta phi_p theta_p', real=True)
f1 = sp.Function('Y_{lm}')(phi,theta)
f2 = sp.Function('Y_{lm}')(phi_p,theta_p)
hb = sp.symbols('hbar')
L,M = sp.symbols('l m', integer=True)


# Ahora vamos a comprobar su funcionalidad aplicando sobre ellos los armonicos esfericos.

def eferic_armonics_symbol(var1,var2,l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l-abs(m))/sp.factorial(l+abs(m)) )
          * sp.exp(sp.I * m * var2) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.Derivative((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(var1))
    return ef

def eferic_armonics_numeric(var1,var2,l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l-abs(m))/sp.factorial(l+abs(m)) )
          * sp.exp(sp.I * m * var2) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.diff((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(var1))
    return ef

def integrate_2d(Z, x, y, method):
    """
    Z: 2D array shaped (len(y), len(x))  -> f(y_i, x_j)
    x: 1D array (len Nx)
    y: 1D array (len Ny)
    method: 'rect', 'trapz', or 'simps'
    """
    if method == 'rect':
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        return np.sum(Z) * dx * dy
    elif method == 'trapz':
        return np.trapezoid(np.trapezoid(Z, x=x, axis=1), x=y, axis=0)
    elif method == 'simps':
        return int.simpson(int.simpson(Z, x=x, axis=1), x=y, axis=0)
    else:
        raise ValueError("method must be 'rect','trapz' or 'simps'")


def safe_lambdify(vars, expr):
    f = sp.lambdify(vars, expr, "numpy")
    return lambda *args: np.broadcast_to(f(*args), np.broadcast(*args).shape)


display (Math(r"Y_{lm}(\theta,\phi) =" + sp.latex(eferic_armonics_symbol(theta,phi,L,M))))
display (Math(r"Y_{lm}(\theta_{p},\phi_{p}) =" + sp.latex(eferic_armonics_symbol(theta_p,phi_p,L,M))))







<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:

# Definimos la multiplicación de los armonicos esfericos para cada marco de coordenadasç



def Y1_Y2  (var1,var2,var1_p,var2_p,l,m,lp,mp):
    mult_esferic = eferic_armonics_numeric(var1,var2,l,m) * eferic_armonics_numeric(var1_p,var2_p,lp,mp)
    return mult_esferic


def Lz_op (var1,var2,f):
    Lz = -sp.I*hb*sp.diff(f,var2)
    return Lz

# Ahora sigamos con el operador L2

def L2_op (var1,var2,f):
    L2 = -hb**2*(sp.diff(f,var1,2)   +  sp.cot(var1)*sp.diff(f,var1)+1/(sp.sin(var1))**2 *sp.diff(f,var2,2))
    return L2


# Ya teniendo los operadores L^2_1, L^2_2, Lz_1, Lz_2 , aplicado sobre el producto de los armónicos esféricos para cada coordenada

# Primero para L2 se tiene que L2(Ylm) = hbar *l*(l+1)Ylm

for l in range(2):
    for m in range(-l,l+1):
        display (Math(r"\hat{L^{2}}"+f"(Y_{{{l}{m}}}"+r"*Y_{l_{p}m_{p}}) =" + sp.latex(sp.trigsimp(sp.nsimplify(sp.simplify(
                                                                        sp.together(
                                                                            sp.trigsimp(
                                                                                sp.powsimp(
                                                                                    sp.cancel(
                                                                                        sp.factor(
                                                                                            sp.expand(L2_op(theta,phi,Y1_Y2(theta,phi,theta_p,phi_p,l,m,L,M)
                                                                                                      )/Y1_Y2(theta,phi,theta_p,phi_p,l,m,L,M))
                                                                                        )
                                                                                    ), force=True
                                                                                )
                                                                            )
                                                                        )
                                                                    ) , rational=True, tolerance=1e-12) )  )))



print("\n")


for l in range(2):
    for m in range(-l,l+1):
        display (Math(r"\hat{L^{2}}"+f"(Y^2_{{{l}{m}}}"+r"*Y^{}_{l_{p}m_{p}}) =" + sp.latex(sp.trigsimp(sp.nsimplify(sp.simplify(
                                                                        sp.together(
                                                                            sp.trigsimp(
                                                                                sp.powsimp(
                                                                                    sp.cancel(
                                                                                        sp.factor(
                                                                                            sp.expand(L2_op(theta_p,phi_p,Y1_Y2(theta,phi,theta_p,phi_p,L,M,l,m)
                                                                                                      )/Y1_Y2(theta,phi,theta_p,phi_p,L,M,l,m))
                                                                                        )
                                                                                    ), force=True
                                                                                )
                                                                            )
                                                                        )
                                                                    ) , rational=True, tolerance=1e-12) )  )))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>